# 1. POS Tagging

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import brown
from nltk import tokenize
import numpy as np
import pandas as pd
import re

In [3]:
import nltk
from nltk.corpus import stopwords
from nltk.corpus import brown
import numpy as np
 
comment=[]
source=pd.read_excel('train.xlsx',encoding='utf-8')
for i in range(len(source)):
    comment.append(source['review_body'][i])

In [4]:
sent_list_raw=[]
sent_list = []
for i in range(len(comment)):
    #去除评论中的表情符号
    comment[i] = str(bytes(comment[i], encoding='utf-8').decode('utf-8').encode('gbk', 'ignore').decode('gbk'))
    #处理评论中的换行符(方便后续分句)
    comment[i] = comment[i].replace('<br />','\n')
    comment[i] = comment[i].replace('<br>','\n')
    #去除乱码
    comment[i] = re.sub('<.*?>','',comment[i])
    
    #将评论（段落）划分为句子
    from nltk import tokenize
    sent_list_raw.append(tokenize.sent_tokenize(comment[i]))

In [5]:
# 使用enchant.checker中的SpellChecker类来解决对评论中的单词进行拼写检查,并进行改正
from enchant.checker import SpellChecker
import enchant

d = enchant.Dict('en_US')
chkr = SpellChecker("en_US")

for s in sent_list_raw:
    wrong_word_list = chkr.set_text(s)
    if wrong_word_list != None:
        for w in wrong_word_list:
            w_right = d.suggest(w)[0]
            s = s.replace(w,w_right)
    sent_list.append(s)

In [169]:
len(sent_list)

6697

In [7]:
#创建一个以一句话为单位的sentence list，并去除其中的标点以供后续处理使用
sentence_list=[]
for i in range(len(sent_list)):
    for j in range(len(sent_list[i])):  
        punctuation = '!,.;:?"\''
        sent_list[i][j] = re.sub(r'[{}]+'.format(punctuation),'',sent_list[i][j])
        sentence_list.append(sent_list[i][j])
        
sentence_list

['I use this phone to maintain and international number',
 'There is no fingerprint reader or facial recognition',
 'It is much better than the last phone I had',
 'The photos are not the best',
 'Solid low cost device',
 'Works as advertised',
 'The SIM card didn’t fit so it kept saying “ enter SIM card “ when it didn’t fit so I couldn’t text anybody even with WiFi and it didn’t have fingerprint unlock also itvwas slow in about the 3 weeks',
 'Battery life was good and arrived with no other problems other then that I couldn’t text or call anyone and it was also in English so I didn’t have any problems with that',
 'I also ended up returning them because I wasn’t that happy with the phone',
 'Great phone my wife use it for work  and love it',
 'Nice phone for everyday use I got the double sim version works great with AT&ampT',
 'A little different to learn',
 'Easy to set up',
 'Garbage',
 'No one can hear me when I call them',
 'Not a good phone @ all',
 'Simple practical and cost-eff

In [8]:
#为每个句子的中的单词标注词性 
reviews_data_raw = []
for i in range(len(sent_list)):
    for text in sent_list[i]:
        text_list = nltk.word_tokenize(text)
        #去掉标点符号（包括换行符）
        english_punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%','/']
        text_list = [word for word in text_list if word not in english_punctuations]
        #使用词性标注器
        sentence=nltk.pos_tag(text_list)
        reviews_data_raw.append(sentence)

reviews_data_raw

[[('I', 'PRP'),
  ('use', 'VBP'),
  ('this', 'DT'),
  ('phone', 'NN'),
  ('to', 'TO'),
  ('maintain', 'VB'),
  ('and', 'CC'),
  ('international', 'JJ'),
  ('number', 'NN')],
 [('There', 'EX'),
  ('is', 'VBZ'),
  ('no', 'DT'),
  ('fingerprint', 'JJ'),
  ('reader', 'NN'),
  ('or', 'CC'),
  ('facial', 'JJ'),
  ('recognition', 'NN')],
 [('It', 'PRP'),
  ('is', 'VBZ'),
  ('much', 'RB'),
  ('better', 'JJR'),
  ('than', 'IN'),
  ('the', 'DT'),
  ('last', 'JJ'),
  ('phone', 'NN'),
  ('I', 'PRP'),
  ('had', 'VBD')],
 [('The', 'DT'),
  ('photos', 'NNS'),
  ('are', 'VBP'),
  ('not', 'RB'),
  ('the', 'DT'),
  ('best', 'JJS')],
 [('Solid', 'NNP'), ('low', 'JJ'), ('cost', 'NN'), ('device', 'NN')],
 [('Works', 'NNS'), ('as', 'IN'), ('advertised', 'JJ')],
 [('The', 'DT'),
  ('SIM', 'NNP'),
  ('card', 'NN'),
  ('didn', 'NN'),
  ('’', 'NNP'),
  ('t', 'NN'),
  ('fit', 'NN'),
  ('so', 'IN'),
  ('it', 'PRP'),
  ('kept', 'VBD'),
  ('saying', 'VBG'),
  ('“', 'JJ'),
  ('enter', 'NN'),
  ('SIM', 'NNP'),
  ('ca

In [9]:
#词干提取
from nltk.stem import WordNetLemmatizer

#词干提取，group为一组单词+词性,返回包含词干的迭代器
def lemmatize_all(group):
    wnl = WordNetLemmatizer()
    word = group[0]
    tag = group[1]
    if tag.startswith('NN'):
        yield wnl.lemmatize(word, pos='n')
    elif tag.startswith('VB'):
        yield wnl.lemmatize(word, pos='v')
    elif tag.startswith('JJ'):
        yield wnl.lemmatize(word, pos='a')
    elif tag.startswith('R'):
        yield wnl.lemmatize(word, pos='r')
    else:
        yield word

reviews_data = []
for sent in reviews_data_raw:
    train_f = []
    for group in sent:
        train_f.append(' '.join(lemmatize_all(group))) #词干
        train_f.append(group[1])#原词的词性
    reviews_data.append(train_f)
    
#将列表中的元素转为元组以供后续分块
reviews_new = []
for s in reviews_data:
    sentence  = []
    i= 0
    while i <len(s)-1:
        l = []
        l.append(s[i])
        l.append(s[i+1])
        l = tuple(l)
        i+=2
        sentence.append(l)
    reviews_new.append(sentence)
    
reviews_new

[[('I', 'PRP'),
  ('use', 'VBP'),
  ('this', 'DT'),
  ('phone', 'NN'),
  ('to', 'TO'),
  ('maintain', 'VB'),
  ('and', 'CC'),
  ('international', 'JJ'),
  ('number', 'NN')],
 [('There', 'EX'),
  ('be', 'VBZ'),
  ('no', 'DT'),
  ('fingerprint', 'JJ'),
  ('reader', 'NN'),
  ('or', 'CC'),
  ('facial', 'JJ'),
  ('recognition', 'NN')],
 [('It', 'PRP'),
  ('be', 'VBZ'),
  ('much', 'RB'),
  ('good', 'JJR'),
  ('than', 'IN'),
  ('the', 'DT'),
  ('last', 'JJ'),
  ('phone', 'NN'),
  ('I', 'PRP'),
  ('have', 'VBD')],
 [('The', 'DT'),
  ('photo', 'NNS'),
  ('be', 'VBP'),
  ('not', 'RB'),
  ('the', 'DT'),
  ('best', 'JJS')],
 [('Solid', 'NNP'), ('low', 'JJ'), ('cost', 'NN'), ('device', 'NN')],
 [('Works', 'NNS'), ('as', 'IN'), ('advertised', 'JJ')],
 [('The', 'DT'),
  ('SIM', 'NNP'),
  ('card', 'NN'),
  ('didn', 'NN'),
  ('’', 'NNP'),
  ('t', 'NN'),
  ('fit', 'NN'),
  ('so', 'IN'),
  ('it', 'PRP'),
  ('keep', 'VBD'),
  ('say', 'VBG'),
  ('“', 'JJ'),
  ('enter', 'NN'),
  ('SIM', 'NNP'),
  ('card', '

In [10]:
#分块
grammer = r'''
NP: {<JJ>+<NN>}
    {<JJ>+<NNS>}
    {<NNP>+<NN>}
    {<NNP>+<NNS>}
    {<NNP>+<NNP>}
    {<NN>+<NN>}
    {<NN>+<NNS>}
    {<NN>+<NNP>}
'''  # 根据词性制定匹配规则
cp = nltk.RegexpParser(grammer) #生成规则
result=[]
for i in range(len(reviews_new)):
    a=cp.parse(reviews_new[i]) #进行分块
    result.append(str(a))
print(result)
# result=cp.parse(reviews_new[1])
# print(result)

['(S\n  I/PRP\n  use/VBP\n  this/DT\n  phone/NN\n  to/TO\n  maintain/VB\n  and/CC\n  (NP international/JJ number/NN))', '(S\n  There/EX\n  be/VBZ\n  no/DT\n  (NP fingerprint/JJ reader/NN)\n  or/CC\n  (NP facial/JJ recognition/NN))', '(S\n  It/PRP\n  be/VBZ\n  much/RB\n  good/JJR\n  than/IN\n  the/DT\n  (NP last/JJ phone/NN)\n  I/PRP\n  have/VBD)', '(S The/DT photo/NNS be/VBP not/RB the/DT best/JJS)', '(S Solid/NNP (NP low/JJ cost/NN) device/NN)', '(S Works/NNS as/IN advertised/JJ)', '(S\n  The/DT\n  (NP SIM/NNP card/NN)\n  didn/NN\n  (NP ’/NNP t/NN)\n  fit/NN\n  so/IN\n  it/PRP\n  keep/VBD\n  say/VBG\n  (NP “/JJ enter/NN)\n  (NP SIM/NNP card/NN)\n  “/NN\n  when/WRB\n  it/PRP\n  didn/VBZ\n  (NP ’/JJ t/JJ fit/NN)\n  so/IN\n  I/PRP\n  couldn/VBP\n  (NP ’/JJ t/NN)\n  (NP text/NN anybody/NN)\n  even/RB\n  with/IN\n  WiFi/NNP\n  and/CC\n  it/PRP\n  didn/VBZ\n  (NP ’/JJ t/NNS)\n  have/VBP\n  (NP fingerprint/NN unlock/NN)\n  also/RB\n  itvwas/JJ\n  slow/VB\n  in/IN\n  about/IN\n  the/DT\n  3/C

In [11]:
#提取句子中的名词
import re
transaction = []

for a in result:
    trans=[]
    #避免正则出现错误
    a = a.replace('(','<')
    a = a.replace(')','>')
    #提取所有名词组块（包含单个名词）中的内容
    noun=[]
    for i in range(len(a.split(' '))):
        if ('NN' in a.split(' ')[i]) and ('NP' not in a.split(' ')[i]):
            noun.append(a.split(' ')[i].strip().replace('/NNS','').replace('>','').replace('/NN',''))
    for word in noun:
        trans.append(word)
            
    phrase = re.findall('<NP (.*?)>',a, re.S)
    
    b = []
    for i in range(len(phrase)):
        #清洗掉词性
        c= phrase[i].replace('/JJ','').replace('/NNS','').replace('/NNP','').replace('/NN','')
        b.append(c)
    for phrase in b:
        trans.append(phrase)
    
    transaction.append(trans)
    
transaction

[['phone', 'number', 'international number'],
 ['reader', 'recognition', 'fingerprint reader', 'facial recognition'],
 ['phone', 'last phone'],
 ['photo'],
 ['cost', 'device', 'low cost'],
 ['Works'],
 ['card',
  'didn',
  't',
  'fit',
  'enter',
  'card',
  '“',
  'fit',
  't',
  'text',
  'anybody',
  't',
  'fingerprint',
  'unlock',
  'week',
  'SIM card',
  '’ t',
  '“ enter',
  'SIM card',
  '’ t fit',
  '’ t',
  'text anybody',
  '’ t',
  'fingerprint unlock'],
 ['life',
  'problem',
  't',
  'text',
  'anyone',
  't',
  'problem',
  'Battery life',
  'other problem',
  '’ t',
  '’ t'],
 ['t', 'phone', '’ t'],
 ['phone', 'wife', 'use', 'work', 'Great phone', 'wife use'],
 ['phone',
  'use',
  'version',
  'ampT',
  'Nice phone',
  'everyday use',
  'double sim version',
  'AT ampT'],
 [],
 [],
 ['Garbage'],
 ['one'],
 ['phone', 'good phone'],
 [],
 ['UI', 'restrictive UI'],
 [],
 ['problem',
  'phone',
  'card',
  'storage',
  'access',
  'memory',
  'apps',
  'SD card',
  'ext

In [13]:
#去掉停用词
stops = set(stopwords.words("english"))
#phone这个词在本例中出现次数过多，删去
stops.add('phone')
stops.add('cell phone')

transaction_file= []
for i in range(len(transaction)):
    sent = []
    for j in range(len(transaction[i])):
        if transaction[i][j] not in stops:
            sent.append(transaction[i][j])
    transaction_file.append(sent)
    
transaction_file

[['number', 'international number'],
 ['reader', 'recognition', 'fingerprint reader', 'facial recognition'],
 ['last phone'],
 ['photo'],
 ['cost', 'device', 'low cost'],
 ['Works'],
 ['card',
  'fit',
  'enter',
  'card',
  '“',
  'fit',
  'text',
  'anybody',
  'fingerprint',
  'unlock',
  'week',
  'SIM card',
  '’ t',
  '“ enter',
  'SIM card',
  '’ t fit',
  '’ t',
  'text anybody',
  '’ t',
  'fingerprint unlock'],
 ['life',
  'problem',
  'text',
  'anyone',
  'problem',
  'Battery life',
  'other problem',
  '’ t',
  '’ t'],
 ['’ t'],
 ['wife', 'use', 'work', 'Great phone', 'wife use'],
 ['use',
  'version',
  'ampT',
  'Nice phone',
  'everyday use',
  'double sim version',
  'AT ampT'],
 [],
 [],
 ['Garbage'],
 ['one'],
 ['good phone'],
 [],
 ['UI', 'restrictive UI'],
 [],
 ['problem',
  'card',
  'storage',
  'access',
  'memory',
  'apps',
  'SD card',
  'external storage',
  'grant access',
  'many apps'],
 ['application',
  'storage',
  'developer',
  'option',
  'externa

# 2. Frequent Features Identification

In [14]:
transaction_df = pd.DataFrame(transaction_file)

# 转换DataFrame数据为包含数据的列表
def deal(data):
	return data.dropna().tolist()
transaction_arr = transaction_df.apply(deal,axis=1).tolist()		        # 方法二

In [15]:
#将数据格式转为可以进行关联分析的格式
from mlxtend.preprocessing import TransactionEncoder	# 传入模型的数据需要满足特定的格式，可以用这种方法来转换为bool值，也可以用函数转换为0、1
 
te = TransactionEncoder()	# 定义模型
transaction_tf = te.fit_transform(transaction_arr)
# df_01 = df_tf.astype('int')			# 将 True、False 转换为 0、1 # 官方给的其它方法
# df_name = te.inverse_transform(df_tf)		# 将编码值再次转化为原来的商品名
transaction_final = pd.DataFrame(transaction_tf,columns=te.columns_)

In [16]:
#提取频繁项集
from mlxtend.frequent_patterns import apriori
 
frequent_itemsets = apriori(transaction_final,min_support=0.01,use_colnames=True)

frequent_itemsets.sort_values(by='support',ascending=False,inplace=True)	# 频繁项集可以按支持度排序
# print(frequent_itemsets[frequent_itemsets.itemsets.apply(lambda x: len(x)) >= 2])  # 选择长度 >=2 的频繁项集

In [17]:
#重置频繁项集列表的索引
frequent_itemsets.reset_index(drop=True,inplace=True)
frequent_itemsets

,support,itemsets
0,0.177438,(abuse)
1,0.177203,(Report abuse)
2,0.177203,"(Report abuse, abuse)"
3,0.038308,(battery)
4,0.029377,(screen)
5,0.025264,(time)
6,0.024383,(day)
7,0.023267,(camera)
8,0.022385,(price)
9,0.021739,(call)


In [18]:
#提取频繁项集中的词组
feature_phrase=[]
for i in range(len(frequent_itemsets['itemsets'])):
    group=[]
    #判断是否是2项集及以上
    if len(frequent_itemsets['itemsets'][i])>=2:
        for word in frequent_itemsets['itemsets'][i]:
            #项集内的元素是词组还是单词，只添加单词
            if ' ' not in word:
                group.append(word)
        if len(group)>=2:
            string=''
            #将二项集拼接成词组
            for i in range(len(group)):
                string=string+group[i]+' '
                
            string=string.strip()
            feature_phrase.append(string)
    
    else:
        for word in frequent_itemsets['itemsets'][i]:
            #寻找feature中的词组
            if ' ' in word:
                feature_phrase.append(word)
                

In [19]:
feature_phrase

['Report abuse', 'life battery']

In [20]:
#提取频繁项集中的单词
feature_noun=[]
for i in range(len(frequent_itemsets['itemsets'])):
    #判断是否是2项集及以上
    if len(frequent_itemsets['itemsets'][i])<2:
        for word in frequent_itemsets['itemsets'][i]:
            #项集内的元素是词组还是单词，只添加单词
            if ' ' not in word:
                feature_noun.append(word)


In [21]:
feature_noun

['abuse',
 'battery',
 'screen',
 'time',
 'day',
 'camera',
 'price',
 'call',
 'card',
 'work',
 'life',
 'cell',
 'quality',
 'feature',
 'problem',
 'product',
 'money',
 'thing',
 'issue',
 'month',
 'service',
 'device',
 'year']

In [22]:
#紧密度剪枝，输入频繁项集中的词组组成的list
def compactness_pruning(phrase_list):
    feature_group=[]
    for phrase in phrase_list:
        num=0
        for i in range(len(sentence_list)):
            if phrase in sentence_list[i]:
                try:
                    #单句中词间隔不超过3
                    if abs(sentence_list[i].split(' ').index(phrase.split(' ')[0])-sentence_list[i].split(' ').index(phrase.split(' ')[1]))<=4:
                        num+=1
                except:
                    continue
        #所有句子中紧密次数大于2
        if num>=2:
            feature_group.append(phrase)
        
    return feature_group #返回所有保留下来的词组

In [53]:
feature_group=compactness_pruning(feature_phrase)
feature_group.remove('Report abuse')
feature_group

['life battery']

In [24]:
# 冗余特征剪枝，输入频繁项集中的单个词汇
def redundancy_pruning(word_list):
    feature_noun=[]
    for word in word_list:
        num=0
        for i in range(len(sentence_list)):
            #feature出现的次数
            if word in sentence_list[i]:
                num+=1
        if num>=3:
            score=0
            #feature是否是其他feature的子集
            for phrase in feature_group:
                if word in phrase:
                    score+=1
            if score==0:
                feature_noun.append(word)
            
    return feature_noun

In [25]:
feature_word=redundancy_pruning(feature_noun)
feature_word

['screen',
 'time',
 'day',
 'camera',
 'price',
 'call',
 'card',
 'work',
 'cell',
 'quality',
 'feature',
 'problem',
 'product',
 'money',
 'thing',
 'issue',
 'month',
 'service',
 'device',
 'year']

# 3. Opinion Words Extraction

In [128]:
#opinion words extraction，输入所有feature的list
def adj_extraction(features):   #把之前提取出来的feature都整到一起
    adjectives={}
    for i in range(len(sentence_list)):
        for feature in features:
            for j in sentence_list[i].split(' '):
                if feature==j:
                    index=sentence_list[i].split(' ').index(feature.split(' ')[0])
                    a=1
                    adj={}
                    #查找距离feature最近的形容词
                    while a<6:
                        #防止索引出界
                        if (index-a>0) and (index-a<len(reviews_data_raw[i])):
                            if reviews_data_raw[i][index-a][-1]=='JJ' or reviews_data_raw[i][index-a][-1]=='JJS':
                                adj[str(i)+'-'+feature]=reviews_data_raw[i][index-a][0]
                                
                        if index+a<len(reviews_data_raw[i]):
                            if reviews_data_raw[i][index+a][-1]=='JJ' or reviews_data_raw[i][index+a][-1]=='JJS':
                                adj[str(i)+'-'+feature]=reviews_data_raw[i][index+a][0]
                                
                        if len(adj)!=0:
                            break
                        a+=1
                            
                            
                    for key in adj.keys():
                        adjectives[key]=adj[key]  
                            
    return adjectives

In [120]:
adj_extraction(feature_group)

{}

In [129]:
adj_dict=adj_extraction(feature_word)
adj_dict

{'4-device': 'low',
 '7-call': '’',
 '19-card': 'external',
 '19-problem': 'biggest',
 '23-problem': 'big',
 '34-screen': 'huge',
 '34-price': 'low',
 '44-time': 'first',
 '63-camera': 'important',
 '64-device': 'new',
 '68-price': 'same',
 '78-problem': 'sure',
 '84-price': 'same',
 '87-price': 'same',
 '91-screen': 'crystal',
 '94-thing': 'same',
 '98-day': 'old',
 '98-work': 'old',
 '110-service': 'great',
 '114-thing': 'only',
 '121-work': 'new',
 '125-card': 'Total',
 '133-year': 'new',
 '135-device': 'prior',
 '141-problem': 'other',
 '142-device': 'expandable',
 '152-time': 'willing',
 '155-price': 'high',
 '160-time': 'valuable',
 '164-year': 'cheaper',
 '165-issue': 'main',
 '181-day': 'other',
 '181-work': 'other',
 '191-price': 'reasonable',
 '192-day': 'prime',
 '193-screen': 'large',
 '193-camera': 'good',
 '193-price': 'good',
 '204-feature': 'intrusive',
 '215-quality': 'fantastic',
 '218-issue': 'fantastic',
 '221-quality': 'sound',
 '232-screen': 'grab',
 '232-quality'

# 4. Orientation for Opinion Words

In [29]:
from nltk.book import *

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


In [35]:
from nltk.corpus import wordnet as wn

In [130]:
#自己创建一个seed_list
seed_list={'good':'pos','nice':'pos','great':'pos','fantastic':'pos','cool':'pos','fine':'pos','well':'pos','fast':'pos',\
           'compatible':'pos','beautiful':'pos','perfect':'pos','clear':'pos','amazing':'pos','awesome':'pos','worthy':'pos',\
          'bad':'neg','dull':'neg','pretty':'pos','annoying':'neg','incompatible':'neg','disappointed':'neg','worthless':'neg',\
           'low':'neg','small':'neg','expensive':'neg','defective':'neg','useless':'neg','awful':'neg','frustrating':'neg',\
           'poor':'neg','excellent':'pos','reliable':'pos','bright':'pos','smart':'pos','difficult':'neg','best':'pos','worst':'neg',\
          'loose':'neg','pleased':'pos','incredible':'pos','favorite':'pos','top':'pos'}

In [131]:
len(seed_list)

42

In [132]:
#seed_list是包含种子形容词及其情感的字典；adj_list是所有opinion word，返回扩充后的seed_list即所有opinionword的情感
def orientation_search(adj_list, seed_list):
    for adj in adj_list:
        #判断形容词是否已经被打过标签
        if adj not in seed_list.keys():
            #提取出词性为形容词的wordnet synset
            synsets=wn.synsets(adj, pos =wn.ADJ)   
            #如果adj在wordnet中有形容词性的词集
            if  synsets !=[]:
                pos=re.findall('\'(.*?)\'',str(synsets[0]))[0]
                #查找seed中有无adj的同义词
                num=0
                for syn in wn.synset(pos).lemma_names():
                    if syn in seed_list:
                        if seed_list[syn]=='pos':
                            seed_list[adj]='pos'
                        else:
                            seed_list[adj]='neg'
                        num+=1
                        
                #如果没有adj的同义词，查找seed中有无反义词
                if num==0:
                    if wn.synset(pos).lemmas()[0].antonyms() !=[]:
                        for ant in wn.synset(pos).lemmas()[0].antonyms()[0].name():
                            if ant in seed_list:
                                if seed_list[ant]=='pos':
                                    seed_list[adj]='neg'
                                else:
                                    seed_list[adj]='pos'
                                    
            #如果adj在wordnet中没有形容词性的词集                        
            else:
                #print(adj+' is not an adjective.')
                continue
        else:
            pass
        
    return seed_list

In [133]:
#执行程序
def orientation_prediction(adj_list,seed_list):
    size1=len(seed_list)
    size2=0
    while size1!=size2:
        size1=len(seed_list)
        orientation_search(adj_list,seed_list)
        size2=len(seed_list)
        
    return seed_list

In [134]:
#情感标注
adj_list=list(adj_dict.values())
# adj_list
adj_orientation=orientation_prediction(adj_list,seed_list)
adj_orientation

{'good': 'pos',
 'nice': 'pos',
 'great': 'pos',
 'fantastic': 'pos',
 'cool': 'pos',
 'fine': 'pos',
 'well': 'pos',
 'fast': 'pos',
 'compatible': 'pos',
 'beautiful': 'pos',
 'perfect': 'pos',
 'clear': 'pos',
 'amazing': 'pos',
 'awesome': 'pos',
 'worthy': 'pos',
 'bad': 'neg',
 'dull': 'neg',
 'pretty': 'pos',
 'annoying': 'neg',
 'incompatible': 'neg',
 'disappointed': 'neg',
 'worthless': 'neg',
 'low': 'neg',
 'small': 'neg',
 'expensive': 'neg',
 'defective': 'neg',
 'useless': 'neg',
 'awful': 'neg',
 'frustrating': 'neg',
 'poor': 'neg',
 'excellent': 'pos',
 'reliable': 'pos',
 'bright': 'pos',
 'smart': 'pos',
 'difficult': 'neg',
 'best': 'pos',
 'worst': 'neg',
 'loose': 'neg',
 'pleased': 'pos',
 'incredible': 'pos',
 'favorite': 'pos',
 'top': 'pos',
 'AMAZING': 'pos',
 'Good': 'pos',
 'frustrated': 'neg',
 'terrible': 'neg',
 'decent': 'pos',
 'ok': 'pos',
 'pesky': 'neg',
 'OK': 'pos',
 'Great': 'pos',
 'hard': 'neg',
 'little': 'neg',
 'Terrible': 'neg',
 'brightes

# 5. Infrequent Feature Identification

In [135]:
#infrequent feature identification，feature_list为特征集，opinion_word为所有形容词，返回非频繁特征集
def infrequent_f_i(feature_list,opinion_words):
    infrequent_feature={}
    for i in range(len(sentence_list)):
        num=0
        for feature in feature_list:
            if feature in sentence_list[i]:
                num+=0
        #如果此句不包含任何特征词
        if num==0:
            for adj in opinion_words:
                for k in sentence_list[i].split(' '):
                    if adj==k:
                        index=sentence_list[i].split(' ').index(adj)
                        a=1
                        feature={}
                        #查找距离形容词最近的名词
                        while a<6:
                            if (index-a>0) and (index-a<len(reviews_data_raw[i])):
                                if reviews_data_raw[i][index-a][-1]=='NN' or reviews_data_raw[i][index-a][-1]=='NNS':
                                    feature[i]=reviews_data_raw[i][index-a][0]
                                    
                            if index+a<len(reviews_data_raw[i]):
                                if reviews_data_raw[i][index+a][-1]=='NN' or reviews_data_raw[i][index+a][-1]=='NNS':
                                    feature[i]=reviews_data_raw[i][index+a][0]
                            if len(feature)!=0:
                                break
                            a+=1
                                
                        for key in feature.keys():
                            infrequent_feature[key]=feature[key]
                        
    return infrequent_feature

In [136]:
#把词组和单词合并成一个feature序列
feature_list=[]
for phrase in feature_group:
    feature_list.append(phrase)
for word in feature_word:
    feature_list.append(word)

opinion_words=list(adj_orientation.keys())
infrequent_features=infrequent_f_i(feature_list,opinion_words)
infrequent_features

{3: 'photos',
 4: 'cost',
 7: 'life',
 9: 'phone',
 10: 'version',
 15: 'phone',
 22: 'selection',
 23: 'ampT',
 25: 'glass',
 27: 'mount',
 34: 'price',
 35: 'things',
 42: 'kids',
 52: 'tier',
 61: 'everything',
 66: 'phone',
 78: 'apps',
 83: 'move',
 91: 'brightness',
 97: 'life',
 101: 'overkill',
 102: 'lot',
 110: 'service',
 119: 'review',
 133: 'cash',
 136: 'reception',
 138: 'way',
 141: 'reception',
 149: 'sensor',
 150: 'research',
 154: 'phone',
 155: 'phone',
 159: 'phone',
 164: 'paper',
 169: 'phone',
 171: 'phone',
 181: 'day',
 185: 'liking',
 186: 'fingerprint',
 191: 'phones',
 193: 'camera',
 196: 'hand',
 199: 'start',
 204: 'feature',
 205: 'spell',
 206: 'button',
 208: 'thats',
 209: 'androids',
 211: 'phone',
 212: 'line',
 214: 'runs',
 215: 'cameras',
 218: 'phone',
 221: 'speaker',
 223: 'port',
 225: 'didnt',
 227: 'calls',
 230: 'screen',
 231: 'storage',
 232: 'fingerprint',
 235: 'brick',
 236: 'phone',
 237: 'battery',
 238: 'camera',
 242: 'phone',
 

# 6.Predicting the Orientations of Opinion Sentences

In [137]:
#把infrequent feature也加到feature_list里
feature_list_final=feature_list
for word in set(infrequent_features.values()):
#     print(word)
    feature_list_final.append(word.strip())
    
feature_list_final=list(set(feature_list_final))
#根据结果反向删除一些出现次数高、不属于手机feature的特征
stop_words=['phone','phones','work','works','use','one','easy','cell phone','cell phones','buy','i','I','day','cell','abuse','get','dont','doesnt','days','cant']
feature_list_final = [x for x in feature_list_final if x not in stop_words]

In [138]:
#提取出所有包含feature的句子
opinion_sentence=[]
op_sentence={}
a=0
for i in range(len(sentence_list)):
    num=0
    features=[]
    for word in sentence_list[i].split(' '):
        for feature in feature_list_final:
            #区分对待phrase和word
            if ' ' in feature:
                if feature in sentence_list[i]:
                    num+=1
                    features.append(feature)
            else:
                if word==feature:
                    num+=1
                    features.append(feature)
    if num>0:
        opinion_sentence.append(sentence_list[i])
        op_sentence[a]=features
        a+=1

In [139]:
opinion_sentence

['I use this phone to maintain and international number',
 'There is no fingerprint reader or facial recognition',
 'The photos are not the best',
 'Solid low cost device',
 'The SIM card didn’t fit so it kept saying “ enter SIM card “ when it didn’t fit so I couldn’t text anybody even with WiFi and it didn’t have fingerprint unlock also itvwas slow in about the 3 weeks',
 'Battery life was good and arrived with no other problems other then that I couldn’t text or call anyone and it was also in English so I didn’t have any problems with that',
 'Great phone my wife use it for work  and love it',
 'Nice phone for everyday use I got the double sim version works great with AT&ampT',
 'Easy to set up',
 'No one can hear me when I call them',
 'The biggest problem with this phone is that it only allows you to use the SD card as external storage but it wont grant access to that memory for many apps that should be able to use it',
 'Even if you enable moving applications to external storage i

In [140]:
op_sentence

{0: ['number'],
 1: ['fingerprint', 'reader', 'recognition'],
 2: ['photos'],
 3: ['low', 'cost', 'device'],
 4: ['SIM',
  'card',
  'fit',
  '“',
  'SIM',
  'card',
  '“',
  'fit',
  'fingerprint',
  'unlock',
  'slow',
  'weeks'],
 5: ['Battery', 'life', 'problems', 'call', 'anyone', 'problems'],
 6: ['wife'],
 7: ['sim', 'version'],
 8: ['set'],
 9: ['call'],
 10: ['problem', 'card', 'storage', 'access', 'memory', 'apps'],
 11: ['storage', 'options', 'support'],
 12: ['keyboard', 'slow'],
 13: ['selection', 'inaccurate'],
 14: ['problem', 'compatible', 'tech'],
 15: ['while', 'date'],
 16: ['screen', 'protector', 'glass', 'feel', 'touch', 'screen'],
 17: ['reasons', 'screen', 'protector'],
 18: ['note', 'fit', 'mount'],
 19: ['buttons'],
 20: ['absolute', 'deal'],
 21: ['paper', 'screen', 'low', 'price'],
 22: ['things', 'experience'],
 23: ['mine', 'flagship', 'model'],
 24: ['pictures', 'kids', 'while'],
 25: ['quality', 'capture'],
 26: ['calls', 'time'],
 27: ['connection'],
 28

# 还原原论文版

In [90]:
#Sentence Orientation,输入所有评论句子和情感词，输出标注每个句子情感的字典
def sentence_orientation(opinion_sentence,opinion_words):
    orientation={-1:'Neutral'}
    for i in range(len(opinion_sentence)):
        score=0
        #判断是否包含从句
        clause=[]
        for word in opinion_sentence[i].split(' '):
            if word=='but':
                clause.append('but')
            elif word=='however':
                clause.append('however')
                
        if 'but' in clause:
            index=opinion_sentence[i].split(' ').index('but')
            op_list=[]
            #首先计算从句中的情感
            for j in opinion_sentence[i].split(' ')[index:]:
                for o in opinion_words:
                    if j==o:
                        op_list.append(o)
                        
            index2=opinion_sentence[i].find('but')
            score+=word_orientation(opinion_sentence[i][index:],op_list)
            #如果从句没有情感倾向
            if orientation==0:
                op_list=[]
                for j in opinion_sentence[i].split(' ')[:index]:
                    for o in opinion_words:
                        if j==o:
                            op_list.append(o)
                score+=word_orientation(opinion_sentence[i][:index],op_list)
                    
        elif 'however'in clause:  #下面就是重复上一步的过程
            index=opinion_sentence[i].split(' ').index('however')
            op_list=[]
            #首先计算从句中的情感
            for j in opinion_sentence[i].split(' ')[index:]:
                for o in opinion_words:
                    if j==o:
                        op_list.append(o)
            score+=word_orientation(opinion_sentence[i][index:],op_list)
            #如果从句没有情感倾向
            if orientation==0:
                op_list=[]
                for j in opinion_sentence[i].split(' ')[:index]:
                    for o in opinion_words:
                        if j==o:
                            op_list.append(o)
                score+=word_orientation(opinion_sentence[i][:index],op_list)
                    
        #如果没有从句
        else:
            op_list=[]
            for j in opinion_sentence[i].split(' '):
                for o in opinion_words:
                    if j==o:
                        op_list.append(o)
            score+=word_orientation(opinion_sentence[i],op_list)
            
        if score>0:
            orientation[i]='Positive'
        elif score<0:
            orientation[i]='Negative'
        #effective opinion orientation
        else:
            score=0
            for feature in feature_list_final:
                for j in opinion_sentence[i].split(' '):
                    if feature==j:
                        index=opinion_sentence[i].split(' ').index(feature)
                        a=1
                        op_word=[]
                        #查找距离feature最近的形容词
                        while a<6:
                            if (index-a>0) and (index-a<len(opinion_sentence[i].split(' '))):
                                if opinion_sentence[i].split(' ')[index-a] in list(adj_orientation.keys()):
                                    op_word.append(opinion_sentence[i].split(' ')[index-a])
                            if index+a<len(opinion_sentence[i].split(' ')):
                                if opinion_sentence[i].split(' ')[index+a] in list(adj_orientation.keys()):
                                    op_word.append(opinion_sentence[i].split(' ')[index-a])
                            if len(op_word)!=0:
                                break
                            a+=1
                                
                        score+=word_orientation(opinion_sentence[i],op_word)
            if score>0:
                orientation[i]='Positive'
            elif score<0:
                orientation[i]='Negative'
            #如果两种方法都不行，用前一句的情感
            else:
                orientation[i]=orientation[i-1]
                    
    return orientation

In [91]:
def word_orientation(sentence, op_list):
    orientation=0
    negation_word=['not','no','yet','neither','seldom']
    for word in op_list:
        try:
            if adj_orientation[word]=='pos':
                index=sentence.split(' ').index(word)
                num=0
                for i in negation_word:
                    if i in sentence.split(' ')[index-6:index-6]:
                        num+=1
                if num>0:
                    orientation-=1
                else:
                    orientation+=1
            else:
                index=sentence.split(' ').index(word)
                num=0
                for i in negation_word:
                    if i in sentence.split(' ')[index-6:index-6]:
                        num+=1
                if num>0:
                    orientation+=1
                else:
                    orientation-=1
        except:
            continue
            
    return orientation

In [92]:
opinion_words=list(adj_orientation.keys())
sentence_orientation=sentence_orientation(opinion_sentence,opinion_words)
sentence_orientation

# 7. Summary Generation

In [93]:
#将结果写到一个表格中
df=pd.DataFrame()
df_feature=[]
df_pos=[]
df_neg=[]
df_pos_sent=[]
df_neg_sent=[]
for feature in feature_list_final:
    df_feature.append(feature)
    pos_num=0
    neg_num=0
    pos_index=[]
    neg_index=[]
    #针对特征的正负情感句子计数
    for i in range(len(opinion_sentence)):
        if feature in op_sentence[i]:
            if sentence_orientation[i]=='Positive':
                pos_num+=1
                pos_index.append(i)
            elif sentence_orientation[i]=='Negative':
                neg_num+=1
                neg_index.append(i)
    df_pos.append(pos_num)
    df_neg.append(neg_num)
    
    #写入正负情感句子
    df_temp=[]
    for i in pos_index:
        df_temp.append(opinion_sentence[i]+'\n')
    df_pos_sent.append(df_temp)
    df_temp=[]
    for i in neg_index:
        df_temp.append(opinion_sentence[i]+'\n')
    df_neg_sent.append(df_temp)

In [94]:
df['Feature']=df_feature
df['Positive Score']=df_pos
df['Negative Score']=df_neg
df['Reviews Number']=df['Positive Score']+df['Negative Score']
df['Average Score']=df['Positive Score']-df['Negative Score']
df['Positive Sentences']=df_pos_sent
df['Negative Sentences']=df_neg_sent

In [95]:
df.sort_values('Reviews Number',ascending=False,na_position='last',inplace=True)
df.reset_index(drop=True,inplace=True)   

In [96]:
df

,Feature,Positive Score,Negative Score,Reviews Number,Average Score,Positive Sentences,Negative Sentences
0,battery,528,89,617,439,[My old A20 had a 4000 mAh battery that frankl...,[Overall performance is exceptional for this p...
1,screen,438,101,539,337,[The phone does not come with a screen protect...,[Out of the box the fingerprint sensor is a li...
2,price,343,30,373,313,[Look this phone is amazing on paper and will ...,[This phone is so beautiful and it could attra...
3,camera,307,36,343,271,[The camera is too important to me among other...,"[Love the camera\n, Cons\n-Very poor camera qu..."
4,time,248,91,339,157,[Had dropped calls for the first time in YEARS...,[You can make the device experience just how y...
...,...,...,...,...,...,...,...
665,isn,0,0,0,0,[],[]
666,ampt,0,0,0,0,[],[]
667,ampT,0,0,0,0,[],[]
668,shortcuts,0,0,0,0,[],[]


In [103]:
df.to_excel('Summary.xlsx',encoding='utf-8',index=False)

# 案例实践版

In [141]:
def sentence_f_o(opinion_sentence, opinion_words):
    orientation={}
    for i in range(len(opinion_sentence)):
        for feature in op_sentence[i]:
            if ' ' in feature:
                index=opinion_sentence[i].split(' ').index(feature.split(' ')[0])
            else:
                index=opinion_sentence[i].split(' ').index(feature)
            a=1
            op_word=[]
            #查找距离feature最近的形容词
            while a<6:
                if (index-a>0) and (index-a<len(opinion_sentence[i].split(' '))):
                    if opinion_sentence[i].split(' ')[index-a] in list(adj_orientation.keys()):
                        op_word.append(opinion_sentence[i].split(' ')[index-a])
                if index+a<len(opinion_sentence[i].split(' ')):
                    if opinion_sentence[i].split(' ')[index+a] in list(adj_orientation.keys()):
                        op_word.append(opinion_sentence[i].split(' ')[index-a])
                if len(op_word)!=0:
                    break
                a+=1
                    
            orientation[str(i)+' '+feature]=word_orientation(opinion_sentence[i],op_word)
            
    return orientation

In [147]:
sentence_f_o=sentence_f_o(opinion_sentence, opinion_words)

In [156]:
sentence_f_o

{'0 number': 0,
 '1 fingerprint': 0,
 '1 reader': 0,
 '1 recognition': 0,
 '2 photos': 0,
 '3 low': 0,
 '3 cost': -1,
 '3 device': -1,
 '4 SIM': 0,
 '4 card': 0,
 '4 fit': 0,
 '4 “': 0,
 '4 fingerprint': 0,
 '4 unlock': 0,
 '4 slow': 0,
 '4 weeks': 0,
 '5 Battery': 0,
 '5 life': 0,
 '5 problems': 0,
 '5 call': 0,
 '5 anyone': 0,
 '6 wife': 0,
 '7 sim': 0,
 '7 version': 0,
 '8 set': 0,
 '9 call': 0,
 '10 problem': 0,
 '10 card': 0,
 '10 storage': 0,
 '10 access': 0,
 '10 memory': 0,
 '10 apps': 0,
 '11 storage': 0,
 '11 options': 0,
 '11 support': 0,
 '12 keyboard': 0,
 '12 slow': 0,
 '13 selection': 0,
 '13 inaccurate': 0,
 '14 problem': 0,
 '14 compatible': 0,
 '14 tech': 0,
 '15 while': 0,
 '15 date': 0,
 '16 screen': 0,
 '16 protector': 0,
 '16 glass': 0,
 '16 feel': 0,
 '16 touch': 1,
 '17 reasons': 0,
 '17 screen': 0,
 '17 protector': 0,
 '18 note': 0,
 '18 fit': 0,
 '18 mount': 1,
 '19 buttons': 0,
 '20 absolute': 0,
 '20 deal': 0,
 '21 paper': 1,
 '21 screen': 0,
 '21 low': 0,
 

In [152]:
df=pd.DataFrame(columns=feature_list_final)
df['opinion sentence']=opinion_sentence
for i in range(len(opinion_sentence)):
    for key in sentence_f_o.keys():
        if str(i) in key:
            df[key.split(' ')[-1]][i]=sentence_f_o[key]

In [154]:
df.fillna(0,inplace=True) 

In [155]:
df

,literate,transmission,duty,unlock,faculties,girls,cable,miui12,minutes,father-in-law,...,steps,math,appput,conditions,deck,surrounding,charger,life battery,transactions,opinion sentence
0,0,-1,0,0,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,I use this phone to maintain and international...
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,1,1,0,0,0,There is no fingerprint reader or facial recog...
2,0,-1,0,0,0,-1,0,1,1,0,...,0,-1,0,0,0,0,0,0,0,The photos are not the best
3,0,0,0,0,0,0,0,0,0,1,...,0,-1,0,1,1,0,0,0,0,Solid low cost device
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,The SIM card didn’t fit so it kept saying “ en...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10539,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Nice item
10540,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Really good product package
10541,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Liked having the prepaid card included
10542,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Great value for the cost


In [166]:
summary=pd.DataFrame(columns=feature_list_final)
summary['reviews']=sent_list
for i in feature_list_final:
    summary[i]=0
for i in range(len(sent_list)):
    for j in range(len(df)):
        if df['opinion sentence'][j] in sent_list[i]:
            for column in feature_list_final:
                summary[column][i]+=df[column][j]

In [168]:
summary.to_excel('Summary.xlsx',encoding='utf-8',index=False)